In [52]:
import torch
import torch.nn as nn
import torch.optim as optim
import math

**多头注意力机制**

参考：https://towardsdatascience.com/build-your-own-transformer-from-scratch-using-pytorch-84c850470dcb

多头注意力机制计算序列中每对位置之间的注意力。它由多个“注意力头”组成，捕捉输入序列的不同方面。

MultiHeadAttention 代码通过输入参数和线性变换层初始化模块。它计算注意力分数，将输入张量切分为多个头，并结合所有头的注意力输出。forward 方法计算多头自注意力，使模型能够关注输入序列的不同方面。

![multi_head_attention](./source/multi_head_attention.png)



In [53]:
# x -> (batch_size,seq_length,d_model)
# option : d_model // num_heads = d_k
# Q,K,V -> (batch_size,seq_length,num_heads,d_k) -> (batch_size,num_heads,seq_length,d_k) 
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        
        self.W_o = nn.Linear(d_model, d_model)
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output
        
    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
        
    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)
        # return x.transpose(1, 2).view(batch_size, seq_length, self.d_model)
        
    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)

        output = self.W_o(self.combine_heads(attn_output))
        return output

**FFN**

PositionWiseFeedForward 类继承了 PyTorch 的 nn.Module，并实现了逐点前馈网络。该类使用两个线性变换层和一个 ReLU 激活函数进行初始化。forward 方法依次应用这些变换和激活函数来计算输出。这个过程使模型在进行预测时能够考虑输入元素的位置。

In [54]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

**PositionalEncoding**

位置编码用于注入输入序列中每个标记的位置信息。它使用不同频率的正弦和余弦函数生成位置编码。


PositionalEncoding 类通过 d_model 和 max_seq_length 输入参数初始化，创建一个张量来存储位置编码值。该类根据缩放因子 div_term 分别计算偶数和奇数索引的正弦和余弦值。forward 方法通过将存储的位置编码值添加到输入张量来计算位置编码，从而使模型能够捕捉输入序列的位置信息。

现在，我们将构建编码器和解码器层。

In [55]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()
        
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        self.register_buffer('pe', pe.unsqueeze(0))
        
    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

**EncoderLayer**

编码器层由一个多头注意力层、一个逐点前馈层和两个层归一化层组成。

EncoderLayer 类通过输入参数和组件初始化，包括一个 MultiHeadAttention 模块、一个 PositionWiseFeedForward 模块、两个层归一化模块和一个 dropout 层。forward 方法通过应用自注意力计算编码器层输出，将注意力输出添加到输入张量并对结果进行归一化。然后，它计算逐点前馈输出，将其与归一化的自注意力输出结合，并在返回处理后的张量之前对最终结果进行归一化。

In [56]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

**DecoderLayer**

解码器层由两个多头注意力层、一个逐点前馈层和三个层归一化层组成。

DecoderLayer 类通过输入参数和组件初始化，包括用于掩码自注意力和交叉注意力的 MultiHeadAttention 模块、PositionWiseFeedForward 模块、三个层归一化模块和一个 dropout 层。


forward 方法通过以下步骤计算解码器层的输出：

* 计算掩码自注意力输出，将其添加到输入张量，接着进行 dropout 和层归一化。
* 计算解码器和编码器输出之间的交叉注意力输出，将其添加到归一化后的掩码自注意力输出，接着进行 dropout 和层归一化。
* 计算逐点前馈输出，将其与归一化后的交叉注意力输出结合，接着进行 dropout 和层归一化。
返回处理后的张量。

这些操作使解码器能够基于输入和编码器输出生成目标序列。


In [57]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x

Transformer

Transformer 类结合了之前定义的模块，创建一个完整的 Transformer 模型。在初始化过程中，Transformer 模块设置输入参数并初始化各种组件，包括用于源序列和目标序列的嵌入层、PositionalEncoding 模块、用于创建堆叠层的 EncoderLayer 和 DecoderLayer 模块、用于投影解码器输出的线性层以及一个 dropout 层。

generate_mask 方法创建源序列和目标序列的二进制掩码，用于忽略填充标记并防止解码器关注未来标记。forward 方法通过以下步骤计算 Transformer 模型的输出：

1. 使用 generate_mask 方法生成源序列和目标序列的掩码。
2. 计算源序列和目标序列的嵌入，并应用位置编码和 dropout。
3. 通过编码器层处理源序列，更新 enc_output 张量。
4. 通过解码器层处理目标序列，使用 enc_output 和掩码，并更新 dec_output 张量。
5. 对解码器输出应用线性投影层，获取输出 logits。

这些步骤使得 Transformer 模型能够根据其组件的联合功能处理输入序列并生成输出序列。

In [58]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        return output

**准备数据**

在本示例中，我们将创建一个玩具数据集进行演示。在实际应用中，您会使用一个更大的数据集，预处理文本，并创建源语言和目标语言的词汇映射。

In [59]:
src_vocab_size = 5000
tgt_vocab_size = 5000
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 100
dropout = 0.1

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

# Generate random sample data
src_data = torch.randint(1, src_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)
tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)

**训练模型**

现在我们将使用示例数据来训练模型。在实际应用中，您会使用一个更大的数据集，并将其分割成训练集和验证集。

In [60]:
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

for epoch in range(10):
    optimizer.zero_grad()
    output = transformer(src_data, tgt_data[:, :-1])
    loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

Epoch: 1, Loss: 8.67525863647461
Epoch: 2, Loss: 8.541637420654297
Epoch: 3, Loss: 8.47097110748291
Epoch: 4, Loss: 8.420844078063965
Epoch: 5, Loss: 8.361783981323242
Epoch: 6, Loss: 8.296092987060547
Epoch: 7, Loss: 8.216520309448242
Epoch: 8, Loss: 8.13842487335205
Epoch: 9, Loss: 8.058088302612305
Epoch: 10, Loss: 7.977389335632324


## 面试问题


### 问题1：为什么采用多头？

在 Transformer 模型中采用多头注意力机制（Multi-Head Attention）的主要原因是它可以增强模型的表达能力和灵活性，使模型能够捕捉到输入序列中的不同特征和关系。以下是具体原因：

1. 捕捉不同的特征和关系

多头注意力机制可以让模型从不同的角度看待输入序列中的信息。每个注意力头会学习不同的子空间表示，从而可以捕捉到序列中的不同模式和依赖关系。通过结合这些不同头的输出，模型可以综合多种视角的信息，获得更丰富的表示。

2. 增强模型的平行计算能力

在多头注意力机制中，输入序列会被分割成多个子空间，并行计算每个子空间的注意力。这种并行计算提高了计算效率，使得训练和推理过程更快。同时，模型可以更好地利用计算资源，提升整体性能。

3. 提高稳定性和性能

多头注意力机制有助于分散注意力，使得模型更稳定，并减少单一头可能带来的噪声和偏差。每个头都能学习到不同的注意力模式，综合起来，可以提高模型的泛化能力和预测性能。

4. 学习长程依赖

在自然语言处理中，词语之间的依赖关系可能跨越较长的距离。多头注意力机制可以让模型在不同头之间关注不同距离的依赖关系，从而更好地捕捉长程依赖信息，提升对上下文的理解能力。


### 问题2：为什么输入需要乘以W矩阵？

在Transformer模型中，输入序列乘以权重矩阵 \( W \) 是为了进行线性变换，从而将输入嵌入到一个高维空间中，使其适应模型的内部表示和处理。具体来说，有以下几个主要原因：

1. 线性变换以适应模型维度
Transformer模型中的输入数据通常是词汇嵌入（word embeddings），这些嵌入的维度可能与模型的内部维度（通常称为 \( d_{model} \)）不匹配。为了将输入嵌入转换成模型所需的维度，必须进行线性变换。这可以通过乘以一个权重矩阵 \( W \) 来实现。假设输入的嵌入维度是 \( d_{input} \)，模型的维度是 \( d_{model} \)，那么权重矩阵 \( W \) 的维度就是 \( d_{input} \times d_{model} \)。

\[ \text{Transformed Input} = \text{Input} \times W \]

2. 参数共享和表示学习
通过引入权重矩阵 \( W \)，模型可以学习到一种有效的表示方法，使得输入序列在高维空间中能够捕捉到更多的语义和上下文信息。这有助于模型更好地理解和处理输入数据。

3. 多头注意力机制中的变换
在多头注意力机制（Multi-Head Attention）中，输入序列会被多次线性变换以生成不同的查询（Query）、键（Key）和值（Value）。具体来说，每个头都会有自己的一组权重矩阵 \( W_Q, W_K, W_V \)，将输入序列分别映射到查询、键和值空间：

\[ Q = XW_Q, \quad K = XW_K, \quad V = XW_V \]

这些线性变换允许每个注意力头捕捉输入数据的不同方面和特征。

1. 增加模型容量
通过在输入上应用线性变换，模型的参数数量增加，从而提高了模型的容量和复杂性。这使得Transformer能够捕捉到更复杂的模式和关系，提高模型的表达能力。

### 问题3：为什么采用scaled dot product Attention机制，而不是additive attention机制？

Transformer模型采用了scaled dot-product attention机制而不是additive attention机制，主要有以下几个原因：

1. 计算效率
- **Scaled Dot-Product Attention**: 这个机制主要依赖于矩阵乘法运算。现代硬件（如GPU和TPU）在处理矩阵乘法时非常高效，可以进行大规模的并行计算。这使得scaled dot-product attention在计算上非常快速。
- **Additive Attention**: 这种机制通常涉及到逐元素的操作和非线性激活函数，计算量较大，无法充分利用矩阵运算的并行性，因此在处理大规模数据时效率较低。

2. 简单性和并行性
- **Scaled Dot-Product Attention**: 公式相对简单，易于实现和优化。其核心步骤是计算查询（Query）和键（Key）之间的点积，然后通过softmax函数归一化。这些操作都可以很自然地并行化。
- **Additive Attention**: 需要先对查询和键进行逐元素加法操作，然后通过一个非线性激活函数（如tanh）处理，最后再乘以一个权重矩阵。这些步骤相对复杂，不容易并行化。

3. 效果和性能
- **Scaled Dot-Product Attention**: 在实际应用中，特别是自然语言处理任务中，scaled dot-product attention表现出色。它被证明在各种任务上都能取得很好的效果，如机器翻译、文本生成等。
- **Additive Attention**: 尽管在某些特定任务中可能表现良好，但整体上其效果往往不如scaled dot-product attention。

4. 规范化和稳定性
- **Scaled Dot-Product Attention**: 在计算查询和键的点积后乘以一个缩放因子 \( \frac{1}{\sqrt{d_k}} \)（其中 \( d_k \) 是键的维度），可以防止随着维度增加点积值变得过大，导致softmax函数输出的梯度过小。这种缩放机制提高了模型的训练稳定性。
- **Additive Attention**: 不存在这种缩放机制，因此在处理高维数据时可能会面临数值稳定性的问题。




### 问题4：为什么scaled dot product Attention机制需要除以sqrt(dk)？

在scaled dot-product attention机制中，除以 \(\sqrt{d_k}\) 的原因主要是为了控制点积结果的幅度，防止其值过大，从而提高数值稳定性。以下是更详细的解释：

1. 点积的幅度控制
当输入向量的维度 \(d_k\) 较大时，查询（Query）和键（Key）向量的点积值也会变得很大。具体来说，如果查询和键向量中的元素服从均值为0、方差为1的正态分布，那么它们的点积的期望值和方差分别为：

\[ \text{E}[\mathbf{q} \cdot \mathbf{k}] = 0 \]
\[ \text{Var}[\mathbf{q} \cdot \mathbf{k}] = d_k \]

这意味着随着 \(d_k\) 的增加，点积的方差会增大，导致点积值可能会变得很大。

2. Softmax函数的梯度消失问题
在attention机制中，点积结果在经过softmax函数处理时，如果值过大，会导致softmax的输出接近于零或一，这样会使梯度在反向传播时几乎消失，进而影响模型的训练效果。具体来说：

- 如果点积结果较大，softmax函数会输出非常尖锐的概率分布，即接近于一的位置概率非常高，其他位置概率非常低。
- 这种情况下，梯度在反向传播时会很小，导致梯度消失问题，模型难以有效训练。

3. 缩放因子的引入
为了缓解上述问题，在计算点积后，我们引入了一个缩放因子 \(\sqrt{d_k}\) 来调整点积值的幅度：

\[ \text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right) V \]

这个缩放因子的作用是将点积值控制在一个较合理的范围内，从而避免softmax函数输出过于尖锐的概率分布，确保梯度在反向传播时不会消失。具体效果是：

- 缩放后的点积值的期望和方差分别为：

\[ \text{E}\left[\frac{\mathbf{q} \cdot \mathbf{k}}{\sqrt{d_k}}\right] = 0 \]
\[ \text{Var}\left[\frac{\mathbf{q} \cdot \mathbf{k}}{\sqrt{d_k}}\right] = 1 \]

这意味着点积结果的分布会更加稳定和均匀，有助于softmax函数产生更合适的概率分布。

问题5：mask机制？